In [1]:
# pip install selenium
# pip install webdriver_manager

In [2]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

import time
import json
import os

## kaggle

- Gather links for each competition

In [3]:
base_url = 'https://www.kaggle.com/competitions?sortOption=reward&page='

# Create a WebDriver instance for Chrome


def kaggle_comp_links(start_page, end_page):
    base_url = 'https://www.kaggle.com/competitions?sortOption=reward&page='
    top_competitions = []

    driver = webdriver.Chrome()
    driver.maximize_window()

    # Note, it seems that we can only scrape 17 pages at a time. Adjust range() to adjust pages you want to scrape
    for page in range(start_page, end_page+1):
        time.sleep(1.5)
        if page == 1:
            url = 'https://www.kaggle.com/competitions?sortOption=reward'  # First page URL
        else:
            url = f'{base_url}{page}'  # Subsequent pages URL
        
        # Visit the page
        driver.get(url)
        
        # Optional: Click all competitions button if needed - assuming this needs to be clicked each time
        if page == 1:
            try:
                button = WebDriverWait(driver, 20).until(
                    EC.element_to_be_clickable((By.XPATH, '//*[@id="site-content"]/div[2]/div/div[4]/div/div[2]/div/div[1]/button[1]'))
                )
                button.click()
            except Exception as e:
                print(f"Failed to click 'All Competitions' button on page {page}: {str(e)}")
        
        # Wait for the links to be visible and collect them
        try:
            competition_links = WebDriverWait(driver, 20).until(
                EC.presence_of_all_elements_located((By.XPATH, '//*[@id="site-content"]/div[2]/div/div[5]/div/div/div/ul/li/div/a'))
            )
            top_competitions.extend([link.get_attribute('href') for link in competition_links])
        except Exception as e:
            print(f"Failed to collect links on page {page}: {str(e)}")
    driver.quit()
    return top_competitions

- Difficulty in scraping all 34 pages at once, split into two part.

In [4]:
top_competitions_1 = kaggle_comp_links(1, 18)
top_competitions_2 = kaggle_comp_links(19, 34)

Failed to collect links on page 31: Message: 



In [5]:
len(top_competitions_1), len(top_competitions_2)

(360, 300)

In [6]:
top_competitions_1[:2]

['https://www.kaggle.com/competitions/ai-mathematical-olympiad-progress-prize-2',
 'https://www.kaggle.com/competitions/passenger-screening-algorithm-challenge']

- Scraping data from each competition

In [8]:
def page_extraction(top_competitions):
    competition_data = []

    for url in top_competitions:
        driver = webdriver.Chrome()
        driver.get(url)
        time.sleep(1)  # Ensure the page loads completely
        
        # Extract competition names
        try:
            competition_name = WebDriverWait(driver, 2).until(
                EC.visibility_of_element_located((By.XPATH, '//*[@id="site-content"]/div[2]/div/div/div[2]/div[2]/div[1]/h1'))
            ).text
        except:
            competition_name = "Competition name not found"

        # Extract the overview text
        try:
            overview_text = WebDriverWait(driver, 2).until(
                EC.visibility_of_element_located((By.XPATH, '//*[@id="abstract"]/div[1]/div[2]/div/p'))
            ).text
        except:
            overview_text = "Overview text not found"

        # Extract all paragraphs in the description section
        try:
            description_paragraphs = WebDriverWait(driver, 2).until(
                EC.presence_of_all_elements_located((By.XPATH, '//*[@id="description"]/div/div[2]/div/div/p'))
            )
            description_text = ' '.join([para.text for para in description_paragraphs])
        except:
            description_text = "Description text not found"

        try:
            driver.get(url + '/data')
            time.sleep(1)
            dataset_paragraphs = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.XPATH, '//*[@id="site-content"]/div[2]/div/div/div[6]/div[1]/div[1]/div/div[2]/div/div[1]/div/div/div/p'))
            )
            dataset_description = ' '.join([para.text for para in dataset_paragraphs])
        except:
            dataset_description = "Dataset description not found"

        # Store the competition url, overview text, and description text
        competition_data.append({
            'name' : competition_name,
            'url': url,
            'overview_text': overview_text,
            'description_text': description_text,
            'dataset_text' : dataset_description
        })
    return competition_data


In [10]:
competition_data_2 = page_extraction(top_competitions_2)
# Output the data
for data in competition_data_2[:5]:
    print(data)

{'name': 'CPROD1: Consumer PRODucts contest #1', 'url': 'https://www.kaggle.com/competitions/cprod1', 'overview_text': 'Overview text not found', 'description_text': "A significant proportion of web usage relates to discussions, research, and purchase of consumer products. Currently, hundreds of thousands of blogs, forums, product review sites, and e-commerce merchants currently exist, in part, to service consumer's need to access product related information and demand to share experiences with products. The goal of this competition is to determine the state-of-the-art methods to automatically recognize product mentions in such textual content and to also disambiguate which product(s) in product catalogs are being referenced. Specifically, the task is to automatically identify all mentions of consumer products in a largely user generated collection of web-content, and to correctly identify the product(s) that each product mention refers to from a large catalog of products. The datasets

In [11]:
output_directory = '../Data'
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Define the output file path
output_file_path = os.path.join(output_directory, 'kaggle_data_first_page_2.json')

# Write the data to a JSON file
with open(output_file_path, 'w') as json_file:
    json.dump(competition_data_2, json_file, indent=4)

print(f"Data successfully written to {output_file_path}")

Data successfully written to ../Data/kaggle_data_first_page_2.json


#### Checking the length

In [12]:
with open('../Data/kaggle_data_first_page_2.json', 'r') as file:
    kaggle_comps = json.load(file)

len(kaggle_comps)

300

## Section 2: Eval.Ai

- Open the major website

In [13]:
website = 'https://eval.ai/web/challenges/list'
# Create a WebDriver instance for Chrome
driver = webdriver.Chrome()
driver.maximize_window()
# Visit the website
driver.get(website)

- Gather the competition webpage links

In [14]:
# Wait for the page to load (this might require adjusting depending on page load time)
driver.implicitly_wait(10)  # Adjust the wait time as necessary

base_xpath = '//*[@id="page-wrap"]/div/div/div/ui-view/ui-view/section/div[2]/div'

# List to hold links
competition_links = []

# Loop through the first two competition divs
for i in range(1, 3):  # Since XPath index starts at 1 and we need first two competitions
    competition_xpath = f'{base_xpath}[{i}]/a'
    # Find the <a> element and get the href attribute
    competition_link = driver.find_element(By.XPATH, competition_xpath).get_attribute('href')
    competition_links.append(competition_link)

# Print the links
for data in competition_links:
    print(data)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="page-wrap"]/div/div/div/ui-view/ui-view/section/div[2]/div[1]/a"}
  (Session info: chrome=137.0.7151.104); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x00000001050ce668 cxxbridge1$str$ptr + 2723108
1   chromedriver                        0x00000001050c68dc cxxbridge1$str$ptr + 2690968
2   chromedriver                        0x0000000104c1a714 cxxbridge1$string$len + 90428
3   chromedriver                        0x0000000104c617c0 cxxbridge1$string$len + 381416
4   chromedriver                        0x0000000104ca2de8 cxxbridge1$string$len + 649232
5   chromedriver                        0x0000000104c559c8 cxxbridge1$string$len + 332784
6   chromedriver                        0x000000010509228c cxxbridge1$str$ptr + 247636

- Scraping from each competition

In [15]:
competition_data_eval = []
for url in competition_links:
    driver.get(url)
    time.sleep(1)  # Ensure the page loads completely
    try:
        paragraphs_xpath = '//*[@id="page-wrap"]/div/div/div/ui-view/ui-view/ui-view/section/div/div[2]/div/div/p'

        # Wait until the presence of all paragraph elements is located
        description_paragraphs = WebDriverWait(driver, 2).until(
            EC.presence_of_all_elements_located((By.XPATH, paragraphs_xpath))
        )

        # Extract text from each paragraph
        competition_overview = ' '.join([paragraph.text for paragraph in description_paragraphs])
    except:
        competition_overview = "Overview text not found"

    try: 
        name_xpath = '//*[@id="page-wrap"]/div/div/div/ui-view/ui-view/section/div/div[1]/div[2]/div/h4'
        competition_name = WebDriverWait(driver, 2).until(
            EC.visibility_of_element_located((By.XPATH, name_xpath))
        ).text
    except:
        name = "Name of competition not found"
    competition_data_eval.append({'url' : url, 
                                'overview' : competition_overview,
                                'name' : competition_name})        

competition_data_eval
    

[]

In [139]:
driver.quit()

## drivendata

In [16]:
website = 'https://www.drivendata.org/competitions/search/?sort=total_prize_purse'
# Create a WebDriver instance for Chrome
driver = webdriver.Chrome()
driver.maximize_window()
# Visit the website
driver.get(website)

In [17]:
try:
    # Wait for the competition list div to load and locate it using its ID
    competition_list_div = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.ID, "competition-list"))
    )

    # Find all <a> tags within nested layers of the competition list div that have the specific class
    competition_links = competition_list_div.find_elements(By.XPATH, ".//a[@class='text-decoration-none'][@href]")

    # Extract href attributes from the first five links only (corrected limit comment)
    hrefs = [link.get_attribute('href') for link in competition_links]  # Limit to first five links
    
    # Output the collected links
    for href in hrefs:
        print(href)
except:
    # Clean up: close the browser window
    hrefs = []

https://www.drivendata.org/competitions/group/nist-federated-learning/
https://www.drivendata.org/competitions/group/nih-nia-alzheimers-adrd-competition/
https://www.drivendata.org/competitions/group/uk-federated-learning/
https://www.drivendata.org/competitions/group/reclamation-water-supply-forecast/
https://www.drivendata.org/competitions/group/competition-reclamation-snow-water/
https://www.drivendata.org/competitions/group/image-similarity-challenge/
https://www.drivendata.org/competitions/group/competition-differential-privacy-deid2/
https://www.drivendata.org/competitions/group/meta-video-similarity/
https://www.drivendata.org/competitions/group/hateful-memes/
https://www.drivendata.org/competitions/group/competition-uva-aiai-challenge/
https://www.drivendata.org/competitions/group/cdc-narratives/
https://www.drivendata.org/competitions/217/cdc-fall-narratives/
https://www.drivendata.org/competitions/63/genetic-engineering-attribution/
https://www.drivendata.org/competitions/gro

In [18]:
hrefs

['https://www.drivendata.org/competitions/group/nist-federated-learning/',
 'https://www.drivendata.org/competitions/group/nih-nia-alzheimers-adrd-competition/',
 'https://www.drivendata.org/competitions/group/uk-federated-learning/',
 'https://www.drivendata.org/competitions/group/reclamation-water-supply-forecast/',
 'https://www.drivendata.org/competitions/group/competition-reclamation-snow-water/',
 'https://www.drivendata.org/competitions/group/image-similarity-challenge/',
 'https://www.drivendata.org/competitions/group/competition-differential-privacy-deid2/',
 'https://www.drivendata.org/competitions/group/meta-video-similarity/',
 'https://www.drivendata.org/competitions/group/hateful-memes/',
 'https://www.drivendata.org/competitions/group/competition-uva-aiai-challenge/',
 'https://www.drivendata.org/competitions/group/cdc-narratives/',
 'https://www.drivendata.org/competitions/217/cdc-fall-narratives/',
 'https://www.drivendata.org/competitions/63/genetic-engineering-attrib

In [ ]:
main_competition_links = hrefs

# Dictionary to hold all sub-competition links for each main competition
all_sub_competition_links = {}

# Iterate over each main competition link
for main_link in main_competition_links:
    driver.get(main_link)
    try:
        # Wait for the sub-competition divs to load
        sub_competition_divs = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "competition-subgroup"))
        )
        # Collect all hrefs from <a> tags within each subgroup
        sub_competition_hrefs = []
        for div in sub_competition_divs:
            sub_competition_links = div.find_elements(By.XPATH, ".//a[@href]")
            for link in sub_competition_links:
                href = link.get_attribute('href')
                if href not in sub_competition_hrefs:
                    sub_competition_hrefs.append(href)

        # Store the collected sub-competition links
        all_sub_competition_links[main_link] = sub_competition_hrefs
    except Exception as e:
        print(f"Error processing {main_link}: {str(e)}")
        all_sub_competition_links[main_link] = []

# Output the collected links for each competition
for main_link, sub_links in all_sub_competition_links.items():
    print(f"Main Competition: {main_link}")
    for link in sub_links:
        print(f"  Sub-Competition: {link}")

In [20]:
# Assuming you have the webdriver and main_competition_links defined
main_competition_links = hrefs

# Dictionary to hold all sub-competition links for each main competition
all_sub_competition_links = {}

# Setup the webdriver
driver = webdriver.Chrome()  # Adjust this line if you use a different browser

# Iterate over each main competition link
for main_link in main_competition_links:
    driver.get(main_link)
    try:
        # Attempt to find sub-competition divs
        sub_competition_divs = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "competition-subgroup"))
        )
        
        # Collect all hrefs from <a> tags within each subgroup
        sub_competition_hrefs = []
        for div in sub_competition_divs:
            sub_competition_links = div.find_elements(By.XPATH, ".//a[@href]")
            for link in sub_competition_links:
                href = link.get_attribute('href')
                if href not in sub_competition_hrefs:
                    sub_competition_hrefs.append(href)

        # Store the collected sub-competition links
        if sub_competition_hrefs:
            all_sub_competition_links[main_link] = sub_competition_hrefs
        else:
            # If no <a> tags are found within subgroups
            all_sub_competition_links[main_link] = [main_link]

    except Exception as e:
        # If error is thrown, assume no sub-competitions are present
        print(f"No sub-competitions found for {main_link}, using main link as sub-competition.")
        all_sub_competition_links[main_link] = [main_link]

# Close the driver
driver.quit()

# Output the collected links for each competition
for main_link, sub_links in all_sub_competition_links.items():
    print(f"Main Competition: {main_link}")
    for link in sub_links:
        print(f"  Sub-Competition: {link}")

No sub-competitions found for https://www.drivendata.org/competitions/217/cdc-fall-narratives/, using main link as sub-competition.
No sub-competitions found for https://www.drivendata.org/competitions/63/genetic-engineering-attribution/, using main link as sub-competition.
No sub-competitions found for https://www.drivendata.org/competitions/78/overhead-geopose-challenge/, using main link as sub-competition.
No sub-competitions found for https://www.drivendata.org/competitions/48/identify-fish-challenge/, using main link as sub-competition.
No sub-competitions found for https://www.drivendata.org/competitions/96/beluga-whales/, using main link as sub-competition.
No sub-competitions found for https://www.drivendata.org/competitions/298/literacy-screening/, using main link as sub-competition.
No sub-competitions found for https://www.drivendata.org/competitions/252/ai-research-assistants/, using main link as sub-competition.
No sub-competitions found for https://www.drivendata.org/comp

In [21]:
list(all_sub_competition_links.items())[-1]
flattened_links = [item for sublist in all_sub_competition_links.values() for item in sublist]

In [22]:
all_sub_competition_links

{'https://www.drivendata.org/competitions/group/nist-federated-learning/': ['https://www.drivendata.org/competitions/98/nist-federated-learning-1/',
  'https://www.drivendata.org/competitions/search/?category=privacy',
  'https://www.drivendata.org/competitions/search/?type=privacy',
  'https://www.drivendata.org/competitions/105/nist-federated-learning-2-financial-crime-federated/',
  'https://www.drivendata.org/competitions/144/nist-federated-learning-2-financial-crime-centralized/',
  'https://www.drivendata.org/competitions/103/nist-federated-learning-2-pandemic-forecasting-federated/',
  'https://www.drivendata.org/competitions/145/nist-federated-learning-2-pandemic-forecasting-centralized/',
  'https://www.drivendata.org/competitions/139/nist-federated-learning-3-red-teams/'],
 'https://www.drivendata.org/competitions/group/nih-nia-alzheimers-adrd-competition/': ['https://www.drivendata.org/competitions/304/prepare-challenge-phase-3/',
  'https://www.drivendata.org/competitions/s

In [23]:
filtered_links = [link for link in flattened_links if 'https://www.drivendata.org/competitions/search/' not in link]

In [24]:
# Setting up Chrome options
options = Options()
options.add_argument("--start-maximized")

# Dictionary to hold the description links
description_links = {}

# Function to scrape description links
def scrape_description_links(link, driver):
    try:
        # Set a page load timeout
        driver.set_page_load_timeout(30)  # Timeout after 30 seconds
        driver.get(link)
        
        # List to hold the description section links
        section_links = []
        # Wait for the li elements to be present, or timeout after 15 seconds
        # lis = WebDriverWait(driver, 15).until(
        #     EC.presence_of_all_elements_located((By.XPATH, '//*[@id="main-container"]/div/div[2]/div[1]/div[2]/ul/li'))
        # )
        lis = WebDriverWait(driver, 15).until(
            EC.presence_of_all_elements_located((By.XPATH, '//*[@id="main-container"]/div/div[2]/div[1]/div[contains(@class, "nav-subpages")]/ul/li'))
        )
        for li in lis:
            a_tag = li.find_element(By.TAG_NAME, 'a')
            href = a_tag.get_attribute('href')
            section_links.append(href)
        
        # Store the list of description links under the corresponding competition link
        description_links[link] = section_links
        return True  # Indicate success
    except TimeoutException:
        print(f"Timeout while processing {link}")
        return False  # Indicate failure
    except Exception as e:
        print(f"Error processing {link}: {e}")
        return False  # Indicate failure

# Set up the WebDriver
driver = webdriver.Chrome(options=options)

# Counters for tracking progress
total_competitions = len(filtered_links)
successful_scrapes = 0
failed_scrapes = 0

# Iterate over all links
for index, link in enumerate(filtered_links, start=1):
    print(f"Processing {index} of {total_competitions}: {link}")
    if scrape_description_links(link, driver):
        successful_scrapes += 1
    else:
        failed_scrapes += 1

driver.quit()

# Report the results
print(f"Completed scraping. Successful: {successful_scrapes}, Failed: {failed_scrapes}")


Processing 1 of 103: https://www.drivendata.org/competitions/98/nist-federated-learning-1/
Processing 2 of 103: https://www.drivendata.org/competitions/105/nist-federated-learning-2-financial-crime-federated/
Processing 3 of 103: https://www.drivendata.org/competitions/144/nist-federated-learning-2-financial-crime-centralized/
Processing 4 of 103: https://www.drivendata.org/competitions/103/nist-federated-learning-2-pandemic-forecasting-federated/
Processing 5 of 103: https://www.drivendata.org/competitions/145/nist-federated-learning-2-pandemic-forecasting-centralized/
Processing 6 of 103: https://www.drivendata.org/competitions/139/nist-federated-learning-3-red-teams/
Processing 7 of 103: https://www.drivendata.org/competitions/304/prepare-challenge-phase-3/
Processing 8 of 103: https://www.drivendata.org/competitions/299/competition-nih-alzheimers-acoustic-2/
Processing 9 of 103: https://www.drivendata.org/competitions/300/competition-nih-alzheimers-sdoh-2/
Processing 10 of 103: htt

In [25]:
description_links

{'https://www.drivendata.org/competitions/98/nist-federated-learning-1/': ['https://www.drivendata.org/competitions/98/nist-federated-learning-1/page/522/',
  'https://www.drivendata.org/competitions/98/nist-federated-learning-1/page/521/',
  'https://www.drivendata.org/competitions/98/nist-federated-learning-1/page/524/',
  'https://www.drivendata.org/competitions/98/nist-federated-learning-1/page/525/',
  'https://www.drivendata.org/competitions/98/nist-federated-learning-1/rules/',
  'https://www.drivendata.org/competitions/98/nist-federated-learning-1/participants/',
  'https://www.drivendata.org/competitions/group/nist-federated-learning/'],
 'https://www.drivendata.org/competitions/105/nist-federated-learning-2-financial-crime-federated/': ['https://www.drivendata.org/competitions/105/nist-federated-learning-2-financial-crime-federated/page/591/',
  'https://www.drivendata.org/competitions/105/nist-federated-learning-2-financial-crime-federated/page/590/',
  'https://www.drivenda

In [27]:
# Dictionary to hold the final text data
final_texts = {}

# Function to scrape description links
def scrape_description_texts(link, driver):
    try:
        driver.set_page_load_timeout(30)
        driver.get(link)

        # Accumulate all texts
        all_texts = []
        # Find the div with class 'prose' and get all direct <p> children
        try:
            prose_div = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div.prose'))
            )
            paragraphs = prose_div.find_elements(By.XPATH, './p')
            for paragraph in paragraphs:
                text = paragraph.text
                if text:  # Check if text is not empty
                    all_texts.append(text)
        except NoSuchElementException:
            print(f"No 'prose' class found in {link}")

        return ' '.join(all_texts)  # Return concatenated text
    except TimeoutException:
        print(f"Timeout while processing {link}")
        return ""
    except Exception as e:
        print(f"Error processing {link}: {e}")
        return ""

# Set up the WebDriver
driver = webdriver.Chrome(options=options)

# Iterate over all competition links and their corresponding description page links
for competition_link, links in description_links.items():
    all_competition_texts = []
    for link in links:
        result_text = scrape_description_texts(link, driver)
        if result_text:
            all_competition_texts.append(result_text)
    # Store concatenated texts for each competition link
    final_texts[competition_link] = ' '.join(all_competition_texts)

driver.quit()

# Print or save the result
print(json.dumps(final_texts, indent=4))

Error processing https://www.drivendata.org/competitions/259/reclamation-water-supply-forecast/data/: name 'NoSuchElementException' is not defined
Error processing https://www.drivendata.org/competitions/262/reclamation-water-supply-forecast-final/data/: name 'NoSuchElementException' is not defined
Error processing https://www.drivendata.org/competitions/257/reclamation-water-supply-forecast-hindcast/data/: name 'NoSuchElementException' is not defined
Error processing https://www.drivendata.org/competitions/254/reclamation-water-supply-forecast-dev/data/: name 'NoSuchElementException' is not defined
Error processing https://www.drivendata.org/competitions/90/competition-reclamation-snow-water-eval/data/: name 'NoSuchElementException' is not defined
Error processing https://www.drivendata.org/competitions/86/competition-reclamation-snow-water-dev/data/: name 'NoSuchElementException' is not defined
Error processing https://www.drivendata.org/competitions/256/pale-blue-dot/data/: name 'No

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [28]:
len(final_texts)

103

In [29]:
file_path = '../Data/drivendata_raw_scraping.json'

# Writing the dictionary to a JSON file
with open(file_path, 'w') as json_file:
    json.dump(final_texts, json_file, indent=4)

In [30]:
with open('./Data', 'w') as json_file:
    json.dump(description_links, json_file, indent=4)

In [31]:
driver.quit()

In [46]:
import json
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

def setup_driver():
    options = Options()
    options.add_argument("--headless")  # optional, remove for debugging
    options.add_argument("--disable-gpu")
    driver = webdriver.Chrome(options=options)
    return driver

def get_text_from_heading(driver, keywords):
    """
    Try to parse div.prose and find text blocks under specified headings.
    """
    try:
        prose_div = driver.find_element(By.CSS_SELECTOR, "div.prose")
        headings = prose_div.find_elements(By.XPATH, ".//h1 | .//h2 | .//h3")
        for heading in headings:
            heading_text = heading.text.strip().lower()
            if any(k in heading_text for k in keywords):
                content_parts = []
                sibling = heading.find_element(By.XPATH, "./following-sibling::*[1]")
                while sibling:
                    if sibling.tag_name in ["h1", "h2", "h3"]:
                        break  # stop at next heading
                    content_parts.append(sibling.text)
                    try:
                        sibling = sibling.find_element(By.XPATH, "./following-sibling::*[1]")
                    except NoSuchElementException:
                        break
                return "\n".join(content_parts)
    except Exception as e:
        print(f"Heading fallback error: {e}")
    return ""


def scrape_competition(driver, url):
    driver.get(url)
    time.sleep(2)

    overview_text = "Competition overview not found"
    dataset_text = "Dataset overview not found"
    evaluation_text = "Evaluation overview not found"

    try:
        # -------- Overview section --------
        sidebar = driver.find_elements(By.CSS_SELECTOR, "div.sidebar.sidebar-toc a")
        overview_link = None
        for link in sidebar:
            if any(x in link.text.lower() for x in ["overview", "background", "about"]):
                overview_link = link
                break
        if overview_link:
            driver.execute_script("arguments[0].scrollIntoView(true);", overview_link)
            time.sleep(0.5)
            overview_link.click()
            time.sleep(1)
            prose = driver.find_element(By.CSS_SELECTOR, "div.prose")
            overview_text = prose.text
        else:
            overview_text = get_text_from_heading(driver, ["overview", "background", "summary", "about"])
        if not overview_text.strip():
            overview_text = "Competition overview not found"

        # -------- Dataset section --------
        right_nav = driver.find_elements(By.CSS_SELECTOR, "div.nav-subpages a")
        data_link = None
        for link in right_nav:
            if any(x in link.text.lower() for x in ["challenge data", "data overview", "open data", "data"]):
                data_link = link
                break
        if data_link:
            driver.execute_script("arguments[0].scrollIntoView(true);", data_link)
            time.sleep(3)
            data_link.click()
            time.sleep(1)
            prose = driver.find_element(By.CSS_SELECTOR, "div.prose")
            dataset_text = prose.text
        else:
            dataset_text = get_text_from_heading(driver, ["data", "dataset", "challenge data", "open data"])
        if not dataset_text.strip():
            dataset_text = "Dataset overview not found"

        # -------- Evaluation section --------
        sidebar = driver.find_elements(By.CSS_SELECTOR, "div.sidebar.sidebar-toc a")
        eval_link = None
        for link in sidebar:
            if any(x in link.text.lower() for x in ["evaluation", "performance metric"]):
                eval_link = link
                break
        if eval_link:
            driver.execute_script("arguments[0].scrollIntoView(true);", eval_link)
            time.sleep(0.5)
            eval_link.click()
            time.sleep(1)
            prose = driver.find_element(By.CSS_SELECTOR, "div.prose")
            evaluation_text = prose.text
        else:
            evaluation_text = get_text_from_heading(driver, ["evaluation", "metric", "performance"])
        if not evaluation_text.strip():
            evaluation_text = "Evaluation overview not found"

    except Exception as e:
        print(f"Error scraping {url}: {e}")

    return {
        "competition_overview": overview_text.strip(),
        "dataset_overview": dataset_text.strip(),
        "evaluation_overview": evaluation_text.strip(),
        "url": url
    }


def main():
    driver = setup_driver()

    with open("../Data/drivendata_urls.json", "r") as f:
        urls_data = json.load(f)

    results = []

    for url in urls_data:
        print(f"Scraping: {url}")
        try:
            result = scrape_competition(driver, url)
            results.append(result)
        except Exception as e:
            print(f"Failed to scrape {url}: {e}")

    driver.quit()

    with open("../Data/drivendata_scrapped_results.json", "w") as f:
        json.dump(results, f, indent=4)

    print("✅ Scraping completed and saved to drivendata_scraped_results.json")

if __name__ == "__main__":
    main()


Scraping: https://www.drivendata.org/competitions/98/nist-federated-learning-1/
Scraping: https://www.drivendata.org/competitions/105/nist-federated-learning-2-financial-crime-federated/
Scraping: https://www.drivendata.org/competitions/144/nist-federated-learning-2-financial-crime-centralized/
Scraping: https://www.drivendata.org/competitions/103/nist-federated-learning-2-pandemic-forecasting-federated/
Scraping: https://www.drivendata.org/competitions/145/nist-federated-learning-2-pandemic-forecasting-centralized/
Scraping: https://www.drivendata.org/competitions/139/nist-federated-learning-3-red-teams/
Scraping: https://www.drivendata.org/competitions/304/prepare-challenge-phase-3/
Scraping: https://www.drivendata.org/competitions/299/competition-nih-alzheimers-acoustic-2/
Scraping: https://www.drivendata.org/competitions/300/competition-nih-alzheimers-sdoh-2/
Scraping: https://www.drivendata.org/competitions/301/prepare-challenge-phase-2-report-arena/
Scraping: https://www.drivenda

## AI CROWD

In [32]:
driver = webdriver.Chrome()
driver.maximize_window()
# Navigate to the website
driver.get('https://www.aicrowd.com/challenges')


In [33]:
# Script to scroll to the bottom of the page

for _ in range(17):
    # Scroll down to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait for a short period to ensure the page has loaded the content
    time.sleep(3)  # Adjust the sleep time if necessary based on the page's response time


In [34]:
links = driver.find_elements(By.XPATH, '//*[@id="challenges-div"]//a[contains(@class, "card-img-overlay")]')
# //*[@id="challenges-div"]/div[1]/div/div[2]/div[1]/h5/a
# Extract href attributes from each link element
urls = [link.get_attribute('href') for link in links]

# Output or process the URLs
for url in urls[:5]:
    print(url)

https://www.aicrowd.com/challenges/commonsense-persona-grounded-dialogue-challenge-2025
https://www.aicrowd.com/challenges/meta-crag-mm-challenge-2025
https://www.aicrowd.com/challenges/brick-by-brick-2024
https://www.aicrowd.com/challenges/sounding-video-generation-svg-challenge-2024
https://www.aicrowd.com/challenges/meta-comprehensive-rag-benchmark-kdd-cup-2024


In [35]:
len(urls)

325

In [38]:
aicrowd_competitions_data = []
driver = webdriver.Chrome()
driver.maximize_window()

for url in urls:
    driver.get(url)
    time.sleep(1)  # Ensure the page loads completely

    # Extract all paragraph texts under the specified div for overview
    paragraphs = driver.find_elements(By.XPATH, '//*[@data-controller="challenge-overview"]//p')
    overview_text = " ".join([p.text for p in paragraphs if p.text])

    # Navigate to the rules page
    nav_links = driver.find_elements(By.XPATH, '//a[contains(@class, "nav-link")]')
    rules_page_url = None
    for link in nav_links:
        if 'rules' in link.get_attribute('href').lower():  # Assuming the URL contains the word 'rules'
            rules_page_url = link.get_attribute('href')
            break

    if rules_page_url:
        driver.get(rules_page_url)
    time.sleep(1)  # Ensure the rules page loads

    # Extract all paragraph texts for rules
    paragraphs = driver.find_elements(By.XPATH, '/html/body/div[2]/main/div[2]/div/div/div/div/p')
    rules_text = " ".join([p.text for p in paragraphs if p.text])

    # Create a dictionary for this competition and append to the list
    competition_data = {
        'url': url,
        'overview': overview_text,
        'rules': rules_text
    }
    aicrowd_competitions_data.append(competition_data)

# Save data to a JSON file
# with open('competitions_data.json', 'w') as file:
#     json.dump(aicrowd_competitions_data, file, indent=4)

ReadTimeoutError: HTTPConnectionPool(host='localhost', port=64233): Read timed out. (read timeout=120)

In [39]:
len(aicrowd_competitions_data)

221

In [40]:
driver.quit()

In [41]:
with open('../Data/aicrowd_raw.json', 'w') as outfile:
        json.dump(aicrowd_competitions_data, outfile, indent=4)